# Analysis of the pisa datasets

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import src.data_lit_covid_school as dlcs
import matplotlib.pyplot as plt
from tueplots.constants.color import rgb

# Load data

In [4]:
def replace_nan_year(local_data: pd.DataFrame, year_lbl: str = "Year") -> pd.DataFrame:
    """Replaces the nan year values with the years of the row before."""
    local_data_c = local_data.copy()
    
    for idx in local_data_c.index:
        current_year = local_data_c.loc[idx][year_lbl]
        if idx == 0:
            assert not np.isnan(current_year), "First year entry should not be none!"
        if np.isnan(current_year):
            local_data_c.loc[idx, year_lbl] = local_data_c.loc[idx - 1][year_lbl]
    
    return local_data_c

def load_sheet(path: str, sheet_name:str, column_map: dict, skiprows=11, skipfooter=5, year_lbl: str = "Year") -> pd.DataFrame:
    """Loads a single sheet of the provided excel"""
    loaded_data = pd.read_excel(path, sheet_name=sheet_name, skiprows=skiprows, skipfooter=skipfooter)
    loaded_data = loaded_data.drop(columns="Unnamed: 0")
    loaded_data = loaded_data.rename(columns=column_map)
    loaded_data = replace_nan_year(loaded_data, year_lbl)
    return loaded_data

def load_subject(path: str, sheet_map: dict, map_all: dict, map_gender: dict, map_repeated: dict, subject: str) -> pd.DataFrame:
    """Loads a complete subject"""
    # Load the single sheets
    average_points = load_sheet(path, sheet_map["average"], map_all)
    gender_points = load_sheet(path, sheet_map["gender"], map_gender)
    repeated_points = load_sheet(path, sheet_map["repeated"], map_repeated)

    # Merge all sheets and add the subject
    merged = pd.merge(average_points, gender_points, on=["Jurisdiction", "Year"])
    merged = pd.merge(merged, repeated_points, on=["Jurisdiction", "Year"])
    merged = merged.assign(Subject=[subject for _ in range(len(merged.index))])
    
    # Melt the structure
    merged = pd.melt(
        merged,
        id_vars=['Year', 'Jurisdiction', 'Subject'],
        var_name='Measure_Type',
        value_name='Value'
    )

    # Extract gender from the measure_gender column
    merged[['Measure', 'Type']] = merged['Measure_Type'].str.split('_', expand=True)
    merged = merged.drop(columns="Measure_Type")
    merged.replace(["—", "†"], np.nan, inplace=True)
    
    return merged
    

In [5]:
FILE_PATH = os.path.join(dlcs.PROJECT_PATH, "data", "pisa", "IDEExcelExport-Dec102023-0927PM.xls")

# Define Maps
col_map_all = {"Year/Study": "Year", "Average": "avg_all", "Standard Error": "std_all"}
col_map_gender = {"Year/Study": "Year", "Average": "avg_male", "Standard Error": "std_male", "Average.1": "avg_female", "Standard Error.1": "std_female"}
col_map_repeated = {"Year/Study": "Year", "Average": "avg_never", "Standard Error": "std_never", "Average.1": "avg_once", "Standard Error.1": "std_once"}

sheet_map_math = {
    "average": "Report 1- Table",
    "gender": "Report 2- Table",
    "repeated": "Report 3- Table",
}
sheet_map_read = {
    "average": "Report 4- Table",
    "gender": "Report 5- Table",
    "repeated": "Report 6- Table",
}
sheet_map_science = {
    "average": "Report 7- Table",
    "gender": "Report 8- Table",
    "repeated": "Report 9- Table",
}

# Load subjects
math = load_subject(FILE_PATH, sheet_map_math, col_map_all, col_map_gender, col_map_repeated, "math")
read = load_subject(FILE_PATH, sheet_map_read, col_map_all, col_map_gender, col_map_repeated, "read")
science = load_subject(FILE_PATH, sheet_map_science, col_map_all, col_map_gender, col_map_repeated, "science")

# Create data frame with all subjects
all_subjects = pd.concat([math, read, science], ignore_index=True).reset_index(drop=True)
all_subjects["Year"] = all_subjects["Year"].astype(int)
all_subjects_clean = all_subjects.dropna()
all_subjects_clean

,Year,Jurisdiction,Subject,Value,Measure,Type
0,2022,International Average (OECD),math,472.358125,avg,all
1,2022,Germany,math,474.826454,avg,all
2,2018,International Average (OECD),math,486.998322,avg,all
3,2018,Germany,math,500.043781,avg,all
4,2015,International Average (OECD),math,484.879397,avg,all
...,...,...,...,...,...,...
447,2018,Germany,science,4.619868,std,once
448,2015,International Average (OECD),science,1.032243,std,once
449,2015,Germany,science,4.666452,std,once
450,2012,International Average (OECD),science,1.20624,std,once
